In [ ]:
# Run module on Google Colab
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
tfe.enable_eager_execution()

import psutil
process = psutil.Process(os.getpid())

In [ ]:
import os
os.chdir("drive/Colab Notebooks/Characters")


data = pd.read_csv("./Data/chars.csv", header=None)
display(data.head())

In [ ]:
# Check image source
i = 1
im = np.asarray(data.iloc[i, 1:], dtype=np.float32).reshape(28,28)
plt.imshow(im)
plt.show()

logits = model(im)
y = tf.argmax(tf.nn.softmax(logits), axis = 1)
print("\nThe Character is:", le.inverse_transform(y.numpy()))

In [ ]:
images = data.iloc[:, 1:]
labels = data.iloc[:, 0]
# fix wrong label
labels[labels == 'y'] = 'Y'

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)
print(le.classes_)
labels = le.transform(labels)

In [ ]:
length = len(labels)
images = np.asarray(images, dtype=np.float32)
labels = np.asarray(labels, dtype=np.int64)

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

In [ ]:
def random_batch(x_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(x_train), batch_size)
    x_batch = x_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return x_batch, y_batch

In [ ]:
# Training Parameters
batch_size = 370
dropout_rate = 0.1

# Network Parameters
height, width = 28, 28
num_classes = 36

In [ ]:
class CNN_Model(tfe.Network):
  def __init__(self):
    super(CNN_Model, self).__init__()
    self.conv1 = self.track_layer(tf.layers.Conv2D(32, 5, activation = tf.nn.relu, padding = "SAME"))
    self.pool1 = self.track_layer(tf.layers.MaxPooling2D(2, 2))
    self.conv2 = self.track_layer(tf.layers.Conv2D(64, 5, activation = tf.nn.relu, padding = "SAME"))
    self.pool2 = self.track_layer(tf.layers.MaxPooling2D(2, 2))
    self.flatten = self.track_layer(tf.layers.Flatten())
    self.fc1 = self.track_layer(tf.layers.Dense(256, activation = tf.nn.relu))
    self.dropout = self.track_layer(tf.layers.Dropout(0.75))
    self.fc2 = self.track_layer(tf.layers.Dense(num_classes, activation = None))
    
  def call(self, input):
    input = tf.reshape(input, [-1, 28, 28, 1])
    result = self.conv1(input)
    result = self.pool1(result)
    result = self.conv2(result)
    result = self.flatten(result)
    result = self.fc1(result)
    result = self.dropout(result)
    result = self.fc2(result)
    return result

In [ ]:
def get_accuracy(logits, y_true, test=False):
  global acc_list
  y_pred = tf.argmax(tf.nn.softmax(logits), axis = 1)
  correct = tf.equal(y_pred, y_true)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#   print(process.memory_info().rss/(10**9))
  if not test:
    acc_list = np.append(acc_list, [[step, accuracy]], axis=0)
    print("step {}:\taccuracy = {}".format(step, accuracy))
  else:
    print("Test accuracy = {}".format(accuracy))  
  
def loss_function(step, model, x, y_true):
  logits = model(x)
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
  loss = tf.reduce_mean(xentropy)
  
  if step % verbose_interval == 0:
    get_accuracy(logits, y_true)
  
  return loss

In [ ]:
# Train model
start = 1
num_steps = 200
verbose_interval = 10
learning_rate = 0.001

acc_list = np.array([[0, 0]])
model = CNN_Model()
optimizer = tf.train.AdamOptimizer(learning_rate)

for step in range(start, num_steps + start):
  batch_x, batch_y = random_batch(x_train, y_train, batch_size)
  batch_x = tf.constant(batch_x)
  batch_y = tf.constant(batch_y)
#   print(batch_y)
  optimizer.minimize(lambda: loss_function(step, model, batch_x, batch_y))

print("Optimization Finished!")
start = start + num_steps

In [ ]:
# Additional run to optimize if needed
learning_rate = 0.0001
steps = 50
verbose_interval = 10
batch_size = 500
optimizer = tf.train.AdamOptimizer(learning_rate)

for step in range(start, start + steps +1):
  batch_x, batch_y = random_batch(x_train, y_train, batch_size)
  batch_x = tf.constant(batch_x)
  batch_y = tf.constant(batch_y)
  optimizer.minimize(lambda: loss_function(step, model, batch_x, batch_y))

start = start + num_steps

In [ ]:
plt.plot(acc_list[:, 0], acc_list[:, 1])
plt.show()

In [ ]:
x_test_batch, y_test_batch = random_batch(x_test, y_test, 10000)
logits = model(x_test_batch)
get_accuracy(logits, y_test_batch, test=True)

In [ ]:
# Remember track_layer in model
# Check variable: model.variables
# tfe.save_network_checkpoint(model,'./Models/cnn_model.ckpt')

In [ ]:
# Run at least 1 round before reconstructing. 
# input = tf.random_normal((1, 784))
# model = CNN_Model()
# model(input) # intitiate variable
# tfe.restore_network_checkpoint(model,'./Models/cnn_model.ckpt')